In [3]:
import cv2
import depthai as dai
import numpy as np
from tensorflow.keras.models import load_model

2024-03-29 17:15:10.308526: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-29 17:15:10.309012: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-29 17:15:10.311750: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-29 17:15:10.355416: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-29 17:15:11.040759: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [4]:
# Load the model
model = load_model('/path/to/image-processing-lab/new_traffic_signs_model.keras')

/home/projects/jupyter/traffic/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 14 variables whereas the saved optimizer has 26 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [5]:
def preprocess(img):
    
    img = cv2.resize(img, (32, 32))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.equalizeHist(img)
    img = img / 255

    return img.reshape(1, 32, 32, 1)

In [6]:
# Create a pipeline
pipeline = dai.Pipeline()

In [5]:
# Configure the camera
camRgb = pipeline.createColorCamera()
camRgb.setPreviewSize(300, 300)
camRgb.setInterleaved(False)
camRgb.setColorOrder(dai.ColorCameraProperties.ColorOrder.BGR)

In [6]:
# XLinkOut: RGB Video Stream
xlinkOut = pipeline.createXLinkOut()
xlinkOut.setStreamName("rgb")
camRgb.preview.link(xlinkOut.input)

In [ ]:
# Connect and start the Pipeline
with dai.Device(pipeline) as device:
    # Output queue
    qRgb = device.getOutputQueue(name="rgb", maxSize=4, blocking=False)

    while True:
        inRgb = qRgb.tryGet()  # Get a Frame 
        
        if inRgb is not None:
            frame = inRgb.getCvFrame()
            processed_frame = preprocess(frame)
            predictions = model.predict(processed_frame)
            predicted_class = np.argmax(predictions, axis=1)
            print("Predicted Class:", predicted_class)

            height, width = frame.shape[:2]  # Dimensions of the pic
            mid_x, mid_y = width // 2, height // 2
            
            # Texteigenschaften
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 1
            font_color = (0, 255, 0)  # Green
            font_thickness = 2
            
            # "Predicted Class" in the middle of the screen
            text_class = "Predicted Sign: "
            text_class_size = cv2.getTextSize(text_class, font, font_scale, font_thickness)[0]
            start_x_class = mid_x - text_class_size[0] // 2
            start_y_class = mid_y - 20  
            
            cv2.putText(frame, text_class, (start_x_class, start_y_class), font, font_scale, font_color, font_thickness)
            
            text_number = f"{str(predicted_class[0])}"  # Number
            start_y_number = start_y_class + text_class_size[1] + 20  # number under the text
            
            cv2.putText(frame, text_number, (start_x_class, start_y_number), font, font_scale, font_color, font_thickness)
            
            # show frame
            cv2.imshow("OAK-D Camera", frame)

        if cv2.waitKey(1) == ord('q'):
            break

cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
Predicted Class: [12]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Predicted Class: [13]


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Predicted Class: [13]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Predicted Class: [12]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Predicted Class: [12]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Predicted Class: [12]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Predicted Class: [12]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Predicted Class: [12]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Predicted Class: [12]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Predicted Class: [12]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Predicted Class: [12]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Predicted Class: [12]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Predicted Class: [13]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Predicted Class: [13]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Predicted Class: [12]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Predicted Class: [12]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Predicted Class: [12]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Predicted Class: [12]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Pr

Only C and default locale supported with the posix collation implementation
Only C and default locale supported with the posix collation implementation
Case insensitive sorting unsupported in the posix collation implementation
Numeric mode unsupported in the posix collation implementation


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Predicted Class: [11]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Predicted Class: [11]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Predicted Class: [11]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Predicted Class: [11]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Predicted Class: [11]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Predicted Class: [11]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Predicted Class: [11]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Predicted Class: [28]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Predicted Class: [28]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Predicted Class: [28]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Predicted Class: [28]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Predicted Class: [28]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Predicted Class: [11]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Predicted Class: [11]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Predicted Class: [28]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Predicted Class: [28]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Pr

Only C and default locale supported with the posix collation implementation
Only C and default locale supported with the posix collation implementation
Case insensitive sorting unsupported in the posix collation implementation
Numeric mode unsupported in the posix collation implementation


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Predicted Class: [40]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Predicted Class: [38]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Predicted Class: [34]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Predicted Class: [34]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Predicted Class: [40]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Predicted Class: [38]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Predicted Class: [40]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Predicted Class: [40]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Predicted Class: [40]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Predicted Class: [40]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Predicted Class: [40]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Predicted Class: [40]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Predicted Class: [28]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Predicted Class: [40]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Predicted Class: [40]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Predicted Class: [40]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Pr